In [5]:
import pandas as pd
import json
from pandas.io.json import json_normalize

In [8]:
FILE = '../logs/running.json'

In [9]:
with open(FILE) as f:
    d = json.load(f)

In [10]:
logs = json_normalize(d)
logs['start_time'] = pd.to_datetime(logs['start_time'])
logs['end_time'] = pd.to_datetime(logs['end_time'])
del logs['losses_val']

In [11]:
logs.head()

,accuracy,accuracy_-1,accuracy_1,end_time,n_workers,running_mode,running_time,start_time
0,0.919620,0.957689,0.876498,2018-05-26 15:30:00,10,synchronous,1262.825377,2018-05-26 15:08:57
1,0.607606,1.000000,0.153774,2018-05-26 15:37:27,10,asynchronous,57.065735,2018-05-26 15:36:30
2,0.628781,1.000000,0.192669,2018-05-26 16:11:49,20,asynchronous,93.332715,2018-05-26 16:10:16
3,0.621003,1.000000,0.199087,2018-05-26 16:21:35,2,asynchronous,27.948042,2018-05-26 16:21:07


In [12]:
times = [pd.DataFrame(d[i]['losses_val']) for i in range(len(d))]

In [13]:
times[0].head()

,loss_val,time
0,2297.214048,2018-05-26 15:09:12
1,2280.627908,2018-05-26 15:09:14
2,2257.971003,2018-05-26 15:09:15
3,2243.362901,2018-05-26 15:09:17
4,2235.283894,2018-05-26 15:09:18
